In [ ]:
!pip install ../input/effnetpytorch/EfficientNet-PyTorch-master/
!pip install ../input/pytorchmetriclearning

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import sklearn
from sklearn.model_selection import train_test_split
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import multiprocessing

from kaggle_secrets import UserSecretsClient

# from skimage.io import imread
import cv2

from skimage.transform import resize
import numpy as np
import math

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

train_df = pd.read_csv("../input/hotel-id-2021-fgvc8/train.csv")
n_classes = len(train_df['hotel_id'].value_counts())

le.fit(train_df['hotel_id'])
train_df['label'] = le.transform(train_df['hotel_id'])
class_map = dict(sorted(train_df[['label', 'hotel_id']].values.tolist()))

In [ ]:

class HotelDataset(torch.utils.data.Dataset):
    """Some Information about CaliforniaDataset"""
    def __init__(self, x_set, y_set, img_size = (224, 224), transform = None, training = True):
        super(HotelDataset, self).__init__()

        self.x_set = x_set
        self.training = training
        if training == True:
            self.y_set = torch.nn.functional.one_hot(y_set)
        self.img_size = img_size
        self.transform = transform

    def __getitem__(self, index):
        #print(self.x_set[index])
        #print(self.x_set[index])
        x = cv2.resize(cv2.imread(self.x_set[index]), dsize = self.img_size)
        #print(x.shape)
        #x = torchvision.transforms.functional.to_tensor(x)
        if self.training == True:
            y = self.y_set[index]
            return (x, y, index)
        if self.transform is not None:
            x = self.transform(x)
        return (x, index)

    def __len__(self):
        return len(self.x_set)

In [ ]:

from efficientnet_pytorch import EfficientNet
from torch import nn
class MLP(nn.Module):
    def __init__(self, layers_size,  final_softmax=False, type = "embedding"):
        super(MLP, self).__init__()
        layers_list = []
        for i in range(1, len(layers_size) - 1):
            layers_list.append(nn.Linear(layers_size[i - 1], layers_size[i]))
            layers_list.append(nn.BatchNorm1d(num_features = layers_size[i]))
            layers_list.append(nn.Tanh())
        layers_list.append(nn.Linear(layers_size[-2], layers_size[-1]))
        if final_softmax:
            layers_list.append(nn.Softmax(dim = 1))
        else:
            layers_list.append(nn.Tanh())
        self.net = nn.Sequential(*layers_list)
    
    def forward(self, x):
        return self.net(x)
        
    

In [ ]:
TEST_DIR = "../input/hotel-id-2021-fgvc8/test_images/"
pretrained_dir = "../input/savedmodelbatchnorm/example_saved_models/"
X_file_name = [name for name in os.listdir(TEST_DIR)]
X_test = [TEST_DIR + name for name in X_file_name]
X_test

In [ ]:
from torchvision import transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set trunk model and replace the softmax layer with an identity function
trunk = EfficientNet.from_name('efficientnet-b0')
trunk_output_size = trunk._fc.out_features

trunk = (trunk.to(device))

# Set embedder model. This takes in the output of the trunk and outputs 64 dimensional embeddings
embedder = (MLP([trunk_output_size, 512, 256]).to(device))

# Set the classifier. The classifier will take the embeddings and output a 50 dimensional vector.
# (Our training set will consist of the first 50 classes of the CIFAR100 dataset.)
# We'll specify the classification loss further down in the code.
classifier = (MLP([256, 256, n_classes], final_softmax = True)).to(device)

# Set optimizers
trunk_optimizer = torch.optim.Adam(trunk.parameters(), lr=0.0001, weight_decay=0.0001)
embedder_optimizer = torch.optim.Adam(embedder.parameters(), lr=0.001, weight_decay=0.0001)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, weight_decay=0.0001)

# Set the image transforms
train_transform = transforms.Compose([transforms.ToTensor(),
                                    #transforms.Resize(224, 224),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transform = transforms.Compose([transforms.ToTensor(),
                                    #transforms.Resize(224, 224),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])





trunk.load_state_dict(torch.load(pretrained_dir + 'trunk_best3.pth'))
embedder.load_state_dict(torch.load(pretrained_dir + 'embedder_best3.pth'))
classifier.load_state_dict(torch.load(pretrained_dir + 'classifier_best3.pth'))



# Set the image transforms
train_transform = transforms.Compose([transforms.ToTensor(),
                                    #transforms.Resize(224, 224),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_transform = transforms.Compose([transforms.ToTensor(),
                                    #transforms.Resize(224, 224),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
testDs = HotelDataset(X_test, None, training = False, transform = val_transform)
testDl = torch.utils.data.DataLoader(
        testDs, 
        batch_size = 1,
        num_workers = 1,
        shuffle=False,
        #sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        #collate_fn=collate_fn,
    )

In [ ]:
def predict(trunk, embedder, classifier, device, val_loader, k = 5):
    s_ls = []
    with torch.no_grad():
        trunk.eval()
        embedder.eval()
        classifier.eval()
        for (x, index) in (val_loader):
            #print(x.shape)
            x = x.to(device)
            label = classifier(embedder(trunk(x)))
            label = torch.squeeze(label)
            values, indices = label.topk(5, dim = 0)
            indices = np.array(indices.to('cpu')).tolist()
            s_ls.append([X_file_name[index], indices])
            #s_ls.append(label)
            #print(label)
    return s_ls
            
            

In [ ]:
cl = predict(trunk, embedder, classifier, device, testDl)


In [ ]:
for v in cl:
    n = []
    for i in v[1]:
        n.append(class_map[i])
    v[1] = " ".join(map(str, n))

In [ ]:
%cd ../working

In [ ]:
pred_df = pd.DataFrame.from_records(cl, columns=['image', 'hotel_id'])
pred_df

In [ ]:
pred_df.to_csv("submission.csv", index=False)

In [ ]:
60291 50996 45167 26181 52458